<a href="https://colab.research.google.com/github/sahinya17/Neural-Network-with-Keras/blob/master/ETC5555_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Read the data and create word embeddings

In [3]:
%%R
lines <- readLines("glove.6B.100d.txt")
embeddings_index <- new.env(hash = TRUE, parent = emptyenv())
for (i in 1:length(lines)) {
  line <- lines[[i]]
  values <- strsplit(line, " ")[[1]]
  word <- values[[1]]
  embeddings_index[[word]] <- as.double(values[-1])
}

## **1. Write a function that calculates the cosine similarity between 2 vector representation of 2 words**

In [10]:
%%R
cosine_similarity <- function(word1_vec, word2_vec){
    
    cosine_similarity = (word1_vec %*% word2_vec) / 
    (sqrt(sum(word1_vec**2)) * sqrt(sum(word2_vec**2)))
    
    return(cosine_similarity)
}

## **2. Calculate the cosine similarity between:**

In [11]:
%%R
# a. “king” and “queen”
king_queen = cosine_similarity(embeddings_index[['king']], 
                               embeddings_index[['queen']])

# b. "king" and "potato"
king_potato = cosine_similarity(embeddings_index[['king']], 
                                embeddings_index[['potato']])

# c. Pick another 2 words and comment if the similarities seem plausible
plant_chair = cosine_similarity(embeddings_index[['plant']], 
                                embeddings_index[['chair']])
plant_leaf = cosine_similarity(embeddings_index[['plant']], 
                               embeddings_index[['leaf']])

print(paste0("king + queen: ", king_queen))
print(paste0("king + potato: ", king_potato))
print(paste0("plant + chair: ", plant_chair))
print(paste0("plant + leaf: ", plant_leaf))

[1] "king + queen: 0.750769079362385"
[1] "king + potato: 0.184115797054578"
[1] "plant + chair: 0.133337478808709"
[1] "plant + leaf: 0.513181511864638"


As it can be observed, words like king and queen and plant and leaf higher word similarities when compared to king and potato and plant and leaf

## **3. Write a function that takes as argument a word embedding vector and finds the closest word within the word embedding. The distance metric is defined again as cosine similarity.**


In [12]:
%%R
find_closest <- function(embeddings_index, word_vec, word_list){
    # Maximum similarity threshold is set to a low value so that 
    # the comparison improves this value
    similarity = -10000

    # Similar word is initially empty
    similar_word = ''

    # Loop through the word embeddings
    for (embed_word in ls(embeddings_index)){
        
        # ensure the word is not part of the inpt word list
        if (!embed_word %in% word_list){
            
            # Get similarity of the 2 words
            current_similarity = cosine_similarity(word_vec, 
                                                embeddings_index[[embed_word]])

            # Check If the current similarity is greater than the 
            # previously set similarity
            if (current_similarity > similarity){
                
                # Replace maximum similarity to be the current similarity
                similarity = current_similarity

                # Set the most similar word to be the current word
                similar_word = embed_word
            }
        }
    }
    return (c(similar_word, similarity))
}

## **4. Calculate the following word embedding operation, using the function you wrote. Comment on the possible bias the result represents.**

In [13]:
%%R
# a. woman - man + king

# Perform operation
word_embed = embeddings_index[['woman']] - 
              embeddings_index[['man']] + 
              embeddings_index[['king']]

# Get closest word and similarity
closest = find_closest(embeddings_index, 
                       word_embed, c('woman', 'man', 'king'))

# Print the closest word and similarity
print(paste0("Closest word to woman - man + king: ", closest[1]))
print(paste0("Cosine similarity: ", closest[2]))

[1] "Closest word to woman - man + king: queen"
[1] "Cosine similarity: 0.783441433588146"


In [14]:
%%R
# b. female - male + doctor

# Perform operation
word_embed = embeddings_index[['female']] - 
              embeddings_index[['male']] + 
              embeddings_index[['doctor']]

# Get closest word and similarity
closest = find_closest(embeddings_index, 
                       word_embed, c('female', 'male', 'doctor'))

# Print the closest word and similarity
print(paste0("Closest word to female - male + doctor: ", closest[1]))
print(paste0("Cosine similarity: ", closest[2]))

[1] "Closest word to female - male + doctor: nurse"
[1] "Cosine similarity: 0.749741727760762"


In [15]:
%%R
# c. female - male + programmer

# Perform operation
word_embed = embeddings_index[['female']] - 
              embeddings_index[['male']] + 
              embeddings_index[['programmer']]

# Get closest word and similarity
closest = find_closest(embeddings_index, 
                       word_embed, c('female', 'male', 'programmer'))

# Print the closest word and similarity
print(paste0("Closest word to female - male + programmer: ", closest[1]))
print(paste0("Cosine similarity: ", closest[2]))

[1] "Closest word to female - male + programmer: animator"
[1] "Cosine similarity: 0.639964123410331"


There seems to be a gender bias in the word embeddings which seem to be evident when looking at the cosine similarities. The word similarity is very less for female programmers, indicating that there are very less places where female programmers are mentioned in the web. Similarly the closest word to women doctor is nurse which indicates bias in women being doctors.  